In [ ]:
import planetary_computer as pc
from pystac_client import Client
import stackstac
import numpy as np
import rasterio
from datetime import datetime, timedelta
import os
import xarray as xr 

# ---------------- PARAMETRY ----------------
# BBOX = [19.03, 49.69, 19.15, 49.78]
BBOX = [16.781111, 50.228889, 16.843611, 50.270556]
EPSG = 32634
RESOLUTION = 20
MAX_CLOUD = 10
NDSI_THRESHOLD = 0.4

START_YEAR = 2015
END_YEAR = datetime.now().year

OUTPUT_DIR = "./ndsi_daily_czarna"
os.makedirs(OUTPUT_DIR, exist_ok=True)
# -------------------------------------------

catalog = Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=pc.sign_inplace,
)

def daterange(start, end):
    for n in range(int((end - start).days) + 1):
        yield start + timedelta(n)

# --- PĘTLA PO SEZONACH ---
for season_year in range(START_YEAR, END_YEAR):
    print(f"\n❄️ Sezon {season_year}/{season_year+1}")

    start_date = datetime(season_year, 11, 1)
    end_date = datetime(season_year + 1, 2, 28)

    for day in daterange(start_date, end_date):

        date_str = day.strftime("%Y-%m-%d")
        print(f" ▶ {date_str}")

        search = catalog.search(
            collections=["sentinel-2-l2a"],
            bbox=BBOX,
            datetime=f"{date_str}/{date_str}",
            query={"eo:cloud_cover": {"lt": MAX_CLOUD}},
        )

        items = list(search.items())
        if not items:
            print("   ⚠️ Brak danych")
            continue

        # --- STACK DLA JEDNEGO DNIA ---
        stack = stackstac.stack(
            items,
            assets=["B03", "B11"],
            resolution=RESOLUTION,
            bounds_latlon=BBOX,
            epsg=EPSG,
            fill_value=np.nan,
        )

        green = stack.sel(band="B03")
        swir = stack.sel(band="B11")

        ndsi = (green - swir) / (green + swir)
        ndsi = ndsi.where(~np.isinf(ndsi))

        # --- JEŚLI WIELE SCEN TEGO DNIA → MEDIANA ---
        ndsi_daily = ndsi.median(dim="time", skipna=True).values

        output_path = f"{OUTPUT_DIR}/ndsi_{date_str}.tif"

        # --- ZAPIS DO XARRAY / NETCDF ---
        da = xr.DataArray(
            ndsi_daily.astype(np.float32),
            dims=("y", "x"),
            name="ndsi",
            attrs={
                "description": "Daily median NDSI",
                "ndsi_threshold": NDSI_THRESHOLD,
                "source": "Sentinel-2 L2A",
                "date": date_str,
            },
        )
        
        ds = xr.Dataset(
            {
                "ndsi": da,
                "snow_mask": (da > NDSI_THRESHOLD).astype("uint8"),
            },
            attrs={
                "bbox": BBOX,
                "epsg": EPSG,
                "resolution": RESOLUTION,
            },
        )
        
        output_path = f"{OUTPUT_DIR}/ndsi_{date_str}.nc"
        ds.to_netcdf(output_path)
        
        print(f"   ✅ zapisano {output_path}")




❄️ Sezon 2015/2016
 ▶ 2015-11-01
   ⚠️ Brak danych
 ▶ 2015-11-02
   ⚠️ Brak danych
 ▶ 2015-11-03
   ⚠️ Brak danych
 ▶ 2015-11-04
   ⚠️ Brak danych
 ▶ 2015-11-05
   ⚠️ Brak danych
 ▶ 2015-11-06
   ⚠️ Brak danych
 ▶ 2015-11-07
   ⚠️ Brak danych
 ▶ 2015-11-08
   ⚠️ Brak danych
 ▶ 2015-11-09
   ⚠️ Brak danych
 ▶ 2015-11-10
   ⚠️ Brak danych
 ▶ 2015-11-11
   ⚠️ Brak danych
 ▶ 2015-11-12
   ⚠️ Brak danych
 ▶ 2015-11-13
   ⚠️ Brak danych
 ▶ 2015-11-14
   ⚠️ Brak danych
 ▶ 2015-11-15
   ⚠️ Brak danych
 ▶ 2015-11-16
   ⚠️ Brak danych
 ▶ 2015-11-17
   ⚠️ Brak danych
 ▶ 2015-11-18
   ⚠️ Brak danych
 ▶ 2015-11-19
   ⚠️ Brak danych
 ▶ 2015-11-20
   ⚠️ Brak danych
 ▶ 2015-11-21
   ⚠️ Brak danych
 ▶ 2015-11-22
   ⚠️ Brak danych
 ▶ 2015-11-23
   ⚠️ Brak danych
 ▶ 2015-11-24
   ⚠️ Brak danych
 ▶ 2015-11-25
   ⚠️ Brak danych
 ▶ 2015-11-26
   ⚠️ Brak danych
 ▶ 2015-11-27
   ⚠️ Brak danych
 ▶ 2015-11-28
   ⚠️ Brak danych
 ▶ 2015-11-29
   ⚠️ Brak danych
 ▶ 2015-11-30
   ⚠️ Brak danych
 ▶ 2015-12-01
   ⚠️ 

ValueError: cannot write NetCDF files because none of the suitable backend libraries (netCDF4, h5netcdf, scipy) are installed

# NDSI Forecasting (Train + Predict)
This section trains a simple regression model on historical daily NDSI GeoTIFFs (ndsi_YYYY-MM-DD.tif) and writes a predicted GeoTIFF for a future date.

Notes:
- Works best when all rasters in the folder have the same grid/shape/CRS.
- The model is a baseline (RandomForest) trained on sampled pixels with lagged NDSI + day-of-year features.
- For “incoming winter season” style forecasting, train with lead=1 and use the rollout command to recursively write predictions over a whole date range.

In [ ]:
from pathlib import Path
import sys
import joblib

# Ensure we can import ndsi_forecast.py regardless of notebook working directory
if Path("ndsi_forecast.py").exists():
    sys.path.insert(0, str(Path(".").resolve()))
elif Path("BITEHACK/ML/ndsi_forecast.py").exists():
    sys.path.insert(0, str(Path("BITEHACK/ML").resolve()))
else:
    raise FileNotFoundError("Can't find ndsi_forecast.py")

from ndsi_forecast import train_model

# Pick one dataset folder with GeoTIFFs named like ndsi_YYYY-MM-DD.tif
DATA_DIR = Path("./ndsi_daily_szczyrk")
if not DATA_DIR.exists():
    DATA_DIR = Path("BITEHACK/ML/ndsi_daily_szczyrk")

MODEL_PATH = Path("./models/ndsi_szczyrk_rf.joblib")

bundle = train_model(
    DATA_DIR,
    lookback=7,   # how many past days to use as input
    lead=7,       # how many days ahead to predict
    sample_pixels_per_day=2000,
    random_state=42,
)

MODEL_PATH.parent.mkdir(parents=True, exist_ok=True)
joblib.dump(bundle, MODEL_PATH)

bundle["metrics"]

In [ ]:
from pathlib import Path
from datetime import date
import sys
import joblib

# Ensure we can import ndsi_forecast.py regardless of notebook working directory
if Path("ndsi_forecast.py").exists():
    sys.path.insert(0, str(Path(".").resolve()))
elif Path("BITEHACK/ML/ndsi_forecast.py").exists():
    sys.path.insert(0, str(Path("BITEHACK/ML").resolve()))
else:
    raise FileNotFoundError("Can't find ndsi_forecast.py")

from ndsi_forecast import predict_to_tif

DATA_DIR = Path("./ndsi_daily_szczyrk")
if not DATA_DIR.exists():
    DATA_DIR = Path("BITEHACK/ML/ndsi_daily_szczyrk")

MODEL_PATH = Path("./models/ndsi_szczyrk_rf.joblib")
bundle = joblib.load(MODEL_PATH)

# If target_date=None -> predicts (last_available_date + lead)
# Or set an explicit date, e.g.: target_date = date(2026, 11, 15)
target_date = None

out_dir = Path("./predictions")
out_dir.mkdir(parents=True, exist_ok=True)

out_path = out_dir / "pred_ndsi.tif"
written = predict_to_tif(bundle, DATA_DIR, target_date=target_date, output_path=out_path)
written